In [1]:
import torch
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers.modeling_bert import BertForMaskedLM

I1215 11:33:16.521343 4744474048 file_utils.py:33] TensorFlow version 2.0.0 available.
I1215 11:33:16.522012 4744474048 file_utils.py:40] PyTorch version 1.3.0.post2 available.


In [2]:
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')

I1215 11:33:18.672933 4744474048 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/cl-tohoku/bert-base-japanese-whole-word-masking-vocab.txt from cache at /Users/m-suzuki/.cache/torch/transformers/ae4d597c0697c617ab6c6c913effc265b7e606830a3b373dfcaeca1794ab9229.5fac9da4d8565963664ed9744688dc7008ff5ec4045f604e9515896f9fe46d9c


In [3]:
model = BertForMaskedLM.from_pretrained('bert-base-japanese-whole-word-masking')

I1215 11:33:20.271379 4744474048 configuration_utils.py:160] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/cl-tohoku/bert-base-japanese-whole-word-masking-config.json from cache at /Users/m-suzuki/.cache/torch/transformers/3c7c1cf64fda50b267a0e88bd23a88a7eaae4e8205f6ee4ceb472ef9e3274f29.3d9f46e22d98b122f65c2c48727301903ca4a5381c3e967e2ba93f0ae64dee75
I1215 11:33:20.272289 4744474048 configuration_utils.py:177] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false

In [4]:
input_ids = tokenizer.encode(f'''
    青葉山で{tokenizer.mask_token}の研究をしています。
''', return_tensors='pt')

In [5]:
print(input_ids)

tensor([[    2, 14211, 28646,    12,     4,     5,   426,    11,    15,    16,
            21,  2610,     8,     3]])


In [6]:
print(tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))

['[CLS]', '青葉', '##山', 'で', '[MASK]', 'の', '研究', 'を', 'し', 'て', 'い', 'ます', '。', '[SEP]']


In [7]:
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
print(masked_index)

4


In [8]:
result = model(input_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))


[CLS] 青葉山 で 植物 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉山 で 温泉 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉山 で 化石 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉山 で 蝶 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉山 で 昆虫 の 研究 を し て い ます 。 [SEP]
